In [22]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds

# Singular Value Decomposition

$$ \underset{(n, d)}A \approx \underset{(n, n)}U * \underset{(n, d)}\Sigma * \underset{(d, d)} V^T  $$

Cualquier matriz de tamaño (n, d) se puede descomponer en producto de tres factores

* En *U* de tamaño (n, n) es una matriz ortogonal que contiene los vectores singulares izquierdos de *A*.
* En $\Sigma$ que es una matriz diagonal (n,d), cuyos valores son los valores singulares de la matriz *A* ordenados en valor decreciente
* En *V* que es una matriz transpuesta (d,d), cuyos valores son los vectores singulares derechos de *A*.

*Ortogonal significa que multiplicando la transpuesta por si misma, se obtiene la matriz identidad*

Con esto lo que se consigue es que podemos ir elminando vectores de las matrices con la información que no es fundamental, (limpiar los datos) y quedarnos con aquella información más determinante.

## Aplicación práctica

Lo que se hace con los motores de recomendación, es para una película que tu no has visto, teniendo en cuenta tus características y las de otros usuarios. Mediante SVD nos quedamos con los usuarios que son parecidos a ti, y vemos las peliculas que no has visto
## Preprocesamos los datos

In [3]:
df_sells = pd.read_csv("retail.csv")
df_sells

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,573744,21314,SMALL GLASS HEART TRINKET POT,8,2011-11-01 08:16:00,2.10,17733.0,United Kingdom
1,573744,21704,BAG 250g SWIRLY MARBLES,12,2011-11-01 08:16:00,0.85,17733.0,United Kingdom
2,573744,21791,VINTAGE HEADS AND TAILS CARD GAME,12,2011-11-01 08:16:00,1.25,17733.0,United Kingdom
3,573744,21892,TRADITIONAL WOODEN CATCH CUP GAME,12,2011-11-01 08:16:00,1.25,17733.0,United Kingdom
4,573744,21915,RED HARMONICA IN BOX,12,2011-11-01 08:16:00,1.25,17733.0,United Kingdom
...,...,...,...,...,...,...,...,...
84706,C579886,22197,POPCORN HOLDER,-1,2011-11-30 17:39:00,0.85,15676.0,United Kingdom
84707,C579886,23146,TRIPLE HOOK ANTIQUE IVORY ROSE,-1,2011-11-30 17:39:00,3.29,15676.0,United Kingdom
84708,C579887,84946,ANTIQUE SILVER T-LIGHT GLASS,-1,2011-11-30 17:42:00,1.25,16717.0,United Kingdom
84709,C579887,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,-1,2011-11-30 17:42:00,7.95,16717.0,United Kingdom


In [43]:
item_dict = dict(zip(df_sells['StockCode'], df_sells['Description']))

In [11]:
sell_matrix = df_sells.pivot_table(
    values='Quantity', 
    index='CustomerID', 
    columns='StockCode',
    aggfunc="sum"
)

In [12]:
sell_matrix

StockCode,10080,10120,10124A,10124G,10125,10135,11001,15030,15034,15036,...,90214M,90214N,90214S,BANK CHARGES,C2,CRUK,D,DOT,M,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12349.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12352.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
12356.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12357.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12362.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18263.0,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18270.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18274.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#Divide in arrays of customers and items for IDs
customer_id_list = np.array(sell_matrix.index.tolist())
item_id_list = np.array(sell_matrix.columns.tolist())

In [18]:
#Create the sparse matrix
sells_mtz = sell_matrix.fillna(0).values.copy()
sells_mtz_sparse = coo_matrix(sells_mtz)

## Realizamos SVD

Obtenemos las tres matrices *U*, sigma y *V*

In [25]:
U, s, V = svds(sells_mtz_sparse, k=10)
s = np.diag(s)

In [27]:
#Unimos las matrices
seels_svd = U @ s @ V

## Creamos la función de recomendación

In [37]:
def recomendar(id_client, num_commends=5):
    # cogemos la fila de la matriz que corresponde a la id de cliente 
    cliente_index = np.where(customer_id_lista == id_client)[0][0]

    # Ordenamos las compras predichas por los clientes en valor descendente
    index_sort = seels_svd[cliente_index, :].argsort()[::-1]

    # creamos una máscara booleana (True/False) de los productos que no ha comprado el cliente
    productos_no_comprados = sells_mtz[cliente_index, :][index_sort] == 0

    rec_index = index_sort[productos_no_comprados]
    rec_ids = item_id_lista[rec_index]
    recomendaciones = rec_ids[:num_commends]
    return recomendaciones

In [53]:
d = sell_matrix.loc[customer_id_list[0]]
[item_dict[x] for x in d[d.notna()].index][:20]

['DOORMAT RED RETROSPOT',
 'SET/5 RED RETROSPOT LID GLASS BOWLS',
 'SET/6 RED SPOTTY PAPER CUPS',
 'PAINTED METAL PEARS ASSORTED',
 'SWEETHEART CERAMIC TRINKET BOX',
 'STRAWBERRY CERAMIC TRINKET POT',
 'GINGHAM HEART  DOORSTOP RED',
 'RED RETROSPOT SUGAR JAM BOWL',
 'RETROSPOT LARGE MILK JUG',
 'RED RETROSPOT SMALL MILK JUG',
 'RED HEART SHAPE LOVE BUCKET ',
 'PINK  HEART SHAPE LOVE BUCKET ',
 'RAIN PONCHO RETROSPOT',
 'CERAMIC STRAWBERRY DESIGN MUG',
 'PINK DOUGHNUT TRINKET POT ',
 'SMALL RED RETROSPOT MUG IN BOX ',
 'SMALL WHITE RETROSPOT MUG IN BOX ',
 'FOOD CONTAINER SET 3 LOVE HEART ',
 'LARGE HEART MEASURING SPOONS',
 'ROUND SNACK BOXES SET OF4 WOODLAND ']

In [59]:
[item_dict[x] for x in recomendar(customer_id_list[0])]

['MINI PAINT SET VINTAGE ',
 'JUMBO BAG RED RETROSPOT',
 'PLACE SETTING WHITE STAR',
 'CHRISTMAS HANGING STAR WITH BELL',
 'CHARLOTTE BAG PINK POLKADOT']